# Route Identification Over Time, Approach 2
See `05_route_identification_over_time.ipynb` for previous work.
<br>Determined that more data is needed than what is given from `fct_monthly_routes`. exploring data from `helpers.import_scheduled_trips` to get feed_key, route short/long name/id/desc and other. 
<br>Cannot rely on feed_key as it is unstable over time. Next best is gtfs_schedule key, but absolute best is source_record_id.
<br>Start with running `make install_env` to install everything needed. `helpers`,`rt_dates`,`sched_gcs`
<br>Query data from warehouse, then use this snippet from `make_routes_gdf` from `_shared_utils/shared_utils/geography_utils.py`.
* ddf["geometry"] = ddf.pt_array.apply(make_linestring)
    
<br> then save out as geo parquet to the `gtfs_schedule` folder in GCS (so versioning and history stays) using 
* utils.geoparquet_gcs_export(
                vp_gdf,
                SEGMENT_GCS,
                f"vp_{analysis_date}"
            )

<br>also add `route_long_name` to query. then see if we are able to see any changes in routes via:
* feed key
* name
* route id
* route short name
* route long name
* route desc
    

In [ ]:
import pandas as pd
#rt_segment_speeds/segment_speed_utils
from segment_speed_utils import helpers

#rt_segment_speeds/segment_speed_utils/project_vars.py
from segment_speed_utils.project_vars import SCHED_GCS, COMPILED_CACHED_VIEWS

#_shared_utils/shared_utils/rt_dates.py
from shared_utils import rt_dates

import geopandas as gpd


In [ ]:
sep_date = rt_dates.DATES["sep2023"]
aug_date = rt_dates.DATES["aug2023"]

# grabbing additional dates
april_date = rt_dates.DATES["apr2023"]
may_date = rt_dates.DATES["may2023"]
june_date = rt_dates.DATES["jun2023"]
july_date = rt_dates.DATES["jul2023"]

In [ ]:
may_date

In [ ]:
# list of all the months
months_list = [sep_date, aug_date, april_date, may_date, june_date, july_date]

In [ ]:
months_list

In [ ]:
# test loop of import_scheduled_trips, iterated through months_list

# empty list
month_trips = {}

for months in months_list:
    df = helpers.import_scheduled_trips(
        months,
        columns=[
            "feed_key",
            "name",
            "gtfs_dataset_key",
            "route_id",
            "route_short_name",
            "route_long_name",
            "route_desc",
        ],
        filters=[[("name", "==", "Sacramento Schedule")]],
        get_pandas=True,
    )
    month_trips[months] = df

In [ ]:
COMPILED_CACHED_VIEWS

In [ ]:
month_trips.keys()

In [ ]:
april = month_trips['2023-04-12'].assign(month='april')
may = month_trips['2023-05-17'].assign(month='may')
june = month_trips['2023-06-14'].assign(month='june')
july = month_trips['2023-07-12'].assign(month='july')
aug = month_trips['2023-08-15'].assign(month='aug')
sep = month_trips['2023-09-13'].assign(month='sep')

In [ ]:
# use this pseudo code to concat all the different months data
# pd.concat can stack DFs on top of others.
# pd.concat([df1, df2], axis=0)

In [ ]:
#test to see if i can use a dictionary in concat
all_trips = pd.concat([april,may,june,july,aug,sep],axis=0)

In [ ]:
all_trips.shape

In [ ]:
all_trips.head()

In [ ]:
# 3 unique feed keys
all_trips.feed_key.value_counts()

In [ ]:
# 2 unique gtfs dataset keys, as opposed to 3 unique feed keys
all_trips.schedule_gtfs_dataset_key.value_counts()

In [ ]:
#expect to see 5 instances of each route id (1 for each month, except june). but some routes only happen 1 time? route 10 and 137
all_trips.route_id.value_counts()

In [ ]:
#similar results with route short name
all_trips.route_short_name.value_counts()

In [ ]:
#further investigation of route 10 and 137, see what their rows say
#only 1 row for each of these routes, both occuring in september
display(all_trips[all_trips['route_id']=='10'])
display(all_trips[all_trips['route_id']=='137'])

In [ ]:
#other routes return multiple rows as expected. 
all_trips[all_trips['route_id']=='F10']

In [ ]:
#would like shape Id and pt array from previous approach, but need a join to make this work
#seek out preliminary joins via metabase


## Amanda's Stuff

In [ ]:
from calitp_data_analysis.tables import tbls
from calitp_data_analysis import utils
from siuba import *

In [ ]:
all_trips.columns

In [ ]:
aprl_sept_2023_routes = gpd.read_parquet("gs://calitp-analytics-data/data-analyses/gtfs_schedule/route_identification_2023_m04_m09.parquet")

In [ ]:
sac = aprl_sept_2023_routes[aprl_sept_2023_routes['name'] == 'Sacramento Schedule']

In [ ]:
sac.sample()

In [ ]:
sac.shape

In [ ]:
sac.source_record_id.value_counts()

* https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.dim_organizations

In [ ]:
dim_gtfs_service_data = (tbls.mart_transit_database.dim_gtfs_service_data()
              >> select( _.source_record_id,
                        _.gtfs_dataset_key,
                        _.name,
                        _._is_current)
              >> filter(_._is_current == True)
              >> collect()
             )

In [ ]:
dim_gtfs_service_data.loc[dim_gtfs_service_data.name.str.contains('Sacramento')]

In [ ]:
sac.sample()

In [ ]:
dim_provider_gtfs_data = (tbls.mart_transit_database.dim_gtfs_service_data()
              >> select( _.organization_source_record_id,
                        _.schedule_gtfs_dataset_key,
                        _.service_name,
                        _._is_current)
              >> filter(_._is_current == True)
              >> collect()
             )

In [ ]:
dim_provider_gtfs_data.shape

In [ ]:
dim_provider_gtfs_data2 = dim_provider_gtfs_data.dropna(subset = ['schedule_gtfs_dataset_key','service_name'])

In [ ]:
dim_provider_gtfs_data2.shape

In [ ]:
# dim_provider_gtfs_data2.service_name.unique()

In [ ]:
dim_provider_gtfs_data2.schedule_gtfs_dataset_key.nunique()

In [ ]:
dim_provider_gtfs_data2.schedule_gtfs_dataset_key.value_counts().head()

* Why are there multiple GTFS keys for one service name??

In [ ]:
dim_provider_gtfs_data2.service_name.value_counts().head()

In [ ]:
dim_provider_gtfs_data2.service_name.value_counts().describe()

In [ ]:
dim_provider_gtfs_data2.loc[dim_provider_gtfs_data2.service_name == "Union City Transit"]

In [ ]:
dim_provider_gtfs_data2.loc[dim_provider_gtfs_data2.service_name.str.contains('Sac')]

In [ ]:
all_trips.shape

In [ ]:
pd.merge(all_trips, dim_provider_gtfs_data2, on = "schedule_gtfs_dataset_key", how = "outer", indicator = True)[['_merge']].value_counts()

In [ ]:
all_trips2 = pd.merge(all_trips, dim_provider_gtfs_data2, on = "schedule_gtfs_dataset_key", how = "inner")

In [ ]:
all_trips2.name.unique()

In [ ]:
all_trips2.organization_source_record_id.unique()

In [ ]:
all_trips2.head()

In [ ]:
dim_provider_gtfs_data.loc[dim_provider_gtfs_data.organization_source_record_id == "recbzZQUIdMmFvm1r"]

In [ ]:
sac.source_record_id.unique()

In [ ]:
pd.merge(all_trips2, sac, left_on = "organization_source_record_id", right_on = "source_record_id", how = "outer", indicator = True)[['_merge']].value_counts()

In [ ]:
pd.merge(all_trips2, sac, on = "name", how = "outer", indicator = True)[['_merge']].value_counts()

In [ ]:
pd.merge(all_trips2, sac, on = "route_id", how = "outer", indicator = True)[['_merge']].value_counts()

In [ ]:
sac.columns

In [ ]:
m1 = pd.merge(all_trips2, sac, left_on = "schedule_gtfs_dataset_key", right_on = ", how = "inner", indicator = True)